In [1]:
%matplotlib inline

In [2]:
import gym
import numpy as np
import random as pr
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

def one_hot(x, depth=16):
    return np.identity(depth)[x:x+1]

Nrandom = 1000
rand_numbers = np.random.rand(Nrandom)
rand_numbers_idx = len(rand_numbers)
def get_rand():
    global rand_numbers, rand_numbers_idx
    if rand_numbers_idx == 0:
        rand_numbers = np.random.rand(Nrandom)
        rand_numbers_idx = len(rand_numbers)
    rand_numbers_idx -= 1
    return rand_numbers[rand_numbers_idx]

def train(env, model, 
          num_episodes=2000, 
          learning_rate=0.1,
          dis=.99):
    rList=[]
    for i in range(num_episodes):
        s = env.reset()
        e = 1.0 / ((i/50)+10)
        rAll = 0
        done = False
        local_loss = []

        Qs_all = []
        x_all = []
        while not done:
            Qs = model.predict(one_hot(s))
            if get_rand() < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Qs)

            s1, reward, done, _ = env.step(a)
            if done:
                Qs[0, a] = reward
            else:
                Qs1 = model.predict(one_hot(s1))
                Qs[0, a] = reward + dis*np.max(Qs1)

            Qs_all.append(Qs[0])
            x_all.append(one_hot(s)[0])

            rAll += reward
            s= s1

        print(f"Episode: {i}")
        model.fit(x=np.array(x_all), y=np.array(Qs_all)) 
        rList.append(rAll)
    return rList

def run(num_episodes=2000):
    env = gym.make("FrozenLake-v0")

    learning_rate = 0.1
    input_size = env.observation_space.n
    output_size = env.action_space.n
    dis = .99

    # input: 16, output: 4, bias: 4 ==> 68
    model = tf.keras.Sequential([
        Dense(output_size, input_shape=[input_size],
             kernel_initializer=tf.random_uniform_initializer(minval=0,maxval=0.01)) 
    ])
    #model.compile(optimizer="sgd", loss='mse')
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse')
    model.summary()
    
    rList = train(env, model, num_episodes=num_episodes,learning_rate=learning_rate,dis=dis)
    
    print("Percent of successful episode: "+str(sum(rList)/num_episodes)+"%")
    plt.bar(range(len(rList)), rList, color='blue')
    plt.show()

In [ ]:
run()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 68        
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________
Episode: 0
1/1 [==============================] - 1s 697ms/step - loss: 1.6636e-06
Episode: 1
1/1 [==============================] - 0s 33ms/step - loss: 3.0026e-06
Episode: 2
1/1 [==============================] - 0s 8ms/step - loss: 5.6355e-06
Episode: 3
2/2 [==============================] - 0s 6ms/step - loss: 1.2470e-06
Episode: 4
1/1 [==============================] - 0s 11ms/step - loss: 6.5043e-06
Episode: 5
1/1 [==============================] - 0s 34ms/step - loss: 1.8737e-06
Episode: 6
1/1 [==============================] - 0s 13ms/step - loss: 9.6920e-07
Episode: 7
1/1 [==============================] - 0s 31ms/step - loss: 9.0854e-06